# SimResults To Wiki Table Utility

In [593]:
import pandas as pd
import numpy as np
import re

### Inputs

##### Series data info

In [594]:
series_directory = "PWC 2020"
race_directory = "hock"
simresults_code = "201222-3Lc"

##### Race format info

In [595]:
table_names = ["Qualify result", "Race 1 result", "Race 2 result"]

##### Table output info

In [596]:
qualy_row_0 = """(% border="1" style="width:554px" %)"""
qualy_row_1 = """|=(% scope="row" style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 41px; text-align: center;" %)Pos|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 44px; text-align: center;" %)No|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 155px; text-align: center;" %)Driver|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 233px; text-align: center;" %)Team|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 77px; text-align: center;" %)Time"""
qualy_row_2 = """|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); text-align: center; width: 41px;" %){}|(% style="border-color:#000000; text-align:center; width:44px" %){}|(% style="border-color:#000000; width:155px" %)[[image:{}||height="14" width="23"]] {}|(% style="text-align:center; border-color:#000000; width:233px" %){}|(% style="text-align:center; border-color:#000000; width:77px" %){}"""
qualy_row = """|=(% style="background-color: rgb(234, 236, 240); text-align: center; width: 41px;" %){}|(% style="text-align:center; width:44px" %){}|(% style="width:155px" %)[[image:{}||height="14" width="23"]] {}|(% style="text-align:center; width:233px" %){}|(% style="text-align:center; width:77px" %){}"""
qualy_row_last = """|=(% colspan="5" style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); text-align: center; width: 552px;" %)[[Source>>{}]]"""

race_row_0 = """(% border="1" style="width:747px" %)"""
race_row_1 = """|=(% scope="row" style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 41px; text-align: center;" %)Pos|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 44px; text-align: center;" %)No|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 155px; text-align: center;" %)Driver|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 229px; text-align: center;" %)Team|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 44px; text-align: center;" %)Laps|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 111px; text-align: center;" %)Time/Retired|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 53px; text-align: center;" %)Grid|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 62px; text-align: center;" %)Points"""
race_row_2 = """|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); text-align: center; width: 41px;" %){}|(% style="border-color:#000000; text-align:center; width:44px" %){}|(% style="border-color:#000000; width:155px" %)[[image:{}||height="14" width="23"]] {}|(% style="text-align:center; border-color:#000000; width:233px" %){}|(% style="text-align:center; border-color:#000000; width:44px" %){}|(% style="text-align:center; border-color:#000000; width:111px" %){}|(% style="text-align:center; border-color:#000000; width:53px" %){}|(% style="text-align:center; border-color:#000000; width:62px" %){}"""
race_row = """|=(% style="background-color: rgb(234, 236, 240); text-align: center; width: 41px;" %){}|(% style="text-align:center; width:44px" %){}|(% style="width:155px" %)[[image:{}||height="14" width="23"]] {}|(% style="text-align:center; width:233px" %){}|(% style="text-align:center; width:44px" %){}|(% style="text-align:center; width:111px" %){}|(% style="text-align:center; width:53px" %){}|(% style="text-align:center; width:62px" %){}"""
race_row_last = """|=(% colspan="8" style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); text-align: center; width: 745px;" %)[[Source>>{}]]"""
race_row_fastestlap = """Fastest lap:  [[image:{}||height="14" width="23"]] {} - {}"""

### Driver and Points Info Parse

In [597]:
drivers_table_file = "{}/drivers_table.csv".format(series_directory)
points_table_file = "{}/points_table.csv".format(series_directory)

results_file = "{}/{}/{}.csv".format(series_directory, race_directory, simresults_code)
output_file = "{}/{}/wiki_tables.txt".format(series_directory, race_directory)

simresults_url = "https://simresults.net/{}".format(simresults_code)

In [598]:
drivers_table = pd.read_csv(drivers_table_file, dtype=object)
display(drivers_table)

,ign,name,number,team,flag
0,McLarenTim,McLarenTim,71,Timberwolves Racing,https://upload.wikimedia.org/wikipedia/en/thum...
1,PhotonBurst,PhotonBurst,28,Timberwolves Racing,https://upload.wikimedia.org/wikipedia/commons...
2,gunciz,Gunciz,000,Phoenix Racing Team Gold,https://upload.wikimedia.org/wikipedia/commons...
3,MrGrinder,MrGrinder,20,Phoenix Racing Team Gold,https://upload.wikimedia.org/wikipedia/en/thum...
4,Trempale,Trempale,17,Monterey SimSport,https://upload.wikimedia.org/wikipedia/en/thum...
5,JEMknight657,JEMknight657,15,Independent,https://upload.wikimedia.org/wikipedia/en/thum...
6,Waffles,Waffles,30,Independent,https://upload.wikimedia.org/wikipedia/en/thum...
7,Salzeder,Salzeder,16,DSW Racing,https://upload.wikimedia.org/wikipedia/en/thum...
8,NotJames,NotJames,73,Monterey SimSport,https://upload.wikimedia.org/wikipedia/en/thum...
9,xPsychedelix,xPsychedelix,14,Phoenix Racing Team Black,https://upload.wikimedia.org/wikipedia/en/thum...


In [599]:
points_table = pd.read_csv(points_table_file, dtype=object)
display(points_table)

,pos,points,qualy_points
0,1,40,3
1,2,36,2
2,3,32,1
3,4,28,0
4,5,25,0
5,6,22,0
6,7,19,0
7,8,16,0
8,9,13,0
9,10,10,0


### Results Tables Parse

In [600]:
rows = {name: [0,0] for name in table_names}

with open(results_file) as fp:
    current_table = ""
    for i, row in enumerate(fp):
        if current_table:
            if row=="\n":
                rows[current_table][1] = i
                print("endeding line {}".format(rows[current_table][1]))
                current_table = ""
        else:
            for name in table_names:
                if row.startswith(name):
                    current_table = name
                    rows[current_table][0] = i+2
                    print("found table '{}' starting line {} ".format(current_table, rows[current_table][0]), end="")

tables = {}

for name in table_names:
    table_skiprows = rows[name][0]
    table_nrows = rows[name][1] - table_skiprows - 1
    table_df = pd.read_csv(results_file, skiprows=table_skiprows, nrows=table_nrows, index_col=False, dtype=object)
    display(table_df)
    tables[name] = table_df

found table 'Qualify result' starting line 15 endeding line 31
found table 'Race 1 result' starting line 319 endeding line 335
found table 'Race 2 result' starting line 745 endeding line 761


,Pos,Class,Team,Vehicle,Driver,Laps,Best lap,Gap
0,1,"""""","""""","""urd_t5_maures_2018""","""McLarenTim""","""6""","""01:28.5970""","""-"""
1,2,"""""","""""","""urd_t5_maures_2018""","""PhotonBurst""","""6""","""01:28.7460""","""00:00.1490"""
2,3,"""""","""""","""urd_t5_aura_2018""","""gunciz""","""5""","""01:29.4850""","""00:00.8880"""
3,4,"""""","""""","""urd_t5_aura_2018""","""MrGrinder""","""4""","""01:29.5850""","""00:00.9880"""
4,5,"""""","""""","""urd_t5_bayro_2018""","""Trempale""","""6""","""01:29.7130""","""00:01.1160"""
5,6,"""""","""""","""urd_t5_bayro_2018""","""JEMknight657""","""5""","""01:30.1900""","""00:01.5930"""
6,7,"""""","""""","""urd_t5_aura_2018""","""Waffles""","""6""","""01:30.2990""","""00:01.7020"""
7,8,"""""","""""","""urd_t5_bayro_2018""","""Salzeder""","""6""","""01:30.3610""","""00:01.7640"""
8,9,"""""","""""","""urd_t5_bayro_2018""","""NotJames""","""7""","""01:30.4320""","""00:01.8350"""
9,10,"""""","""""","""urd_t5_aura_2018""","""xPsychedelix""","""6""","""01:30.4640""","""00:01.8670"""


,Pos,Class,Team,Vehicle,Driver,Laps,Time/Retired,Best lap,Consistency,Led,Pits,Unnamed: 11
0,1,"""""","""""","""urd_t5_maures_2018""","""McLarenTim""","""15""",""" 22:28.3460""","""01:29.1690""","""99.77%""","""14""","""0""",
1,2,"""""","""""","""urd_t5_maures_2018""","""PhotonBurst""","""15""",""" '+00:01.3470""","""01:29.1110""","""99.70%""","""0""","""0""",
2,3,"""""","""""","""urd_t5_aura_2018""","""gunciz""","""15""",""" '+00:18.0710""","""01:30.0140""","""99.60%""","""0""","""0""",
3,4,"""""","""""","""urd_t5_aura_2018""","""MrGrinder""","""15""",""" '+00:18.8660""","""01:29.7000""","""99.24%""","""0""","""0""",
4,5,"""""","""""","""urd_t5_bayro_2018""","""JEMknight657""","""15""",""" '+00:19.4820""","""01:29.6630""","""99.35%""","""0""","""0""",
5,6,"""""","""""","""urd_t5_aura_2018""","""Waffles""","""15""",""" '+00:28.8730""","""01:30.0270""","""98.95%""","""0""","""0""",
6,7,"""""","""""","""urd_t5_aura_2018""","""gman197002""","""15""",""" '+00:37.5820""","""01:30.8640""","""99.40%""","""0""","""0""",
7,8,"""""","""""","""urd_t5_bayro_2018""","""Salzeder""","""15""",""" '+00:38.5890""","""01:30.5520""","""99.01%""","""0""","""0""",
8,9,"""""","""""","""urd_t5_bayro_2018""","""Trempale""","""15""",""" '+00:44.5450""","""01:29.8730""","""97.31%""","""0""","""0""",
9,10,"""""","""""","""urd_t5_aura_2018""","""xPsychedelix""","""15""",""" '+01:00.1450""","""01:31.1270""","""97.91%""","""0""","""0""",


,Pos,Class,Team,Vehicle,Driver,Laps,Time/Retired,Best lap,Consistency,Led,Pits,Unnamed: 11
0,1,"""""","""""","""urd_t5_maures_2018""","""McLarenTim""","""28""",""" 42:14.7630""","""01:29.1040""","""98.64%""","""23""","""0""",
1,2,"""""","""""","""urd_t5_maures_2018""","""PhotonBurst""","""28""",""" '+00:28.3980""","""01:29.1570""","""98.72%""","""4""","""0""",
2,3,"""""","""""","""urd_t5_aura_2018""","""MrGrinder""","""28""",""" '+00:31.8870""","""01:29.6750""","""98.04%""","""0""","""0""",
3,4,"""""","""""","""urd_t5_bayro_2018""","""Trempale""","""28""",""" '+00:41.4270""","""01:30.1590""","""98.46%""","""0""","""0""",
4,5,"""""","""""","""urd_t5_bayro_2018""","""JEMknight657""","""28""",""" '+00:41.8710""","""01:29.8220""","""98.20%""","""0""","""0""",
5,6,"""""","""""","""urd_t5_aura_2018""","""Waffles""","""28""",""" '+00:50.8330""","""01:30.4620""","""98.27%""","""0""","""0""",
6,7,"""""","""""","""urd_t5_aura_2018""","""gman197002""","""28""",""" '+01:09.5980""","""01:30.9060""","""98.19%""","""0""","""0""",
7,8,"""""","""""","""urd_t5_bayro_2018""","""Salzeder""","""28""",""" '+01:10.4410""","""01:30.7750""","""97.99%""","""0""","""0""",
8,9,"""""","""""","""urd_t5_bayro_2018""","""NotJames""","""28""",""" '+01:11.5500""","""01:30.8460""","""98.36%""","""0""","""0""",
9,10,"""""","""""","""urd_t5_aura_2018""","""xPsychedelix""","""28""",""" '+01:13.0400""","""01:30.2530""","""97.97%""","""0""","""0""",


### Results Table Clean

In [601]:
for name, table_df in tables.items():
    
    table_df = table_df.apply(lambda s: s.str.strip(' \'"'), axis=1)
    table_df = table_df.rename(columns=lambda c: c.strip(' \'"'))
    table_df = table_df.apply(lambda s: s.str.replace('(.*\d\d*\.\d{3})0$', r'\1'), axis=1)
    
    table_df["Pos"] = table_df["Pos"].astype(int)
    table_df["Laps"] = table_df["Laps"].astype(int)
    
    table_df = table_df[table_df["Laps"] > 0]
    
    table_df = table_df[table_df.columns.drop(list(table_df.filter(regex='Unnamed*')))]
    table_df = table_df.dropna(how='all', axis='columns')
    
    tables[name] = table_df

def merge_qualy_info(race_table, qualy_table, add_qualy_points=False):
    if add_qualy_points:
        qualy_table = qualy_table[["Driver", "Pos", "Points"]]
        qualy_table = qualy_table.rename(columns={"Pos": "Grid", "Points": "Qualify Points"})
    else:
        qualy_table = qualy_table[["Driver", "Pos"]]
        qualy_table = qualy_table.rename(columns={"Pos": "Grid"})
        
    race_table = race_table.merge(qualy_table, how='left', on="Driver")

    race_table["Grid"] = race_table["Grid"].fillna(-1)
    race_table["Grid"] = race_table["Grid"].astype(int)
    race_table["Pos"] = race_table["Pos"].astype(int)
    race_table["Laps"] = race_table["Laps"].astype(int)
    
    if add_qualy_points:
        race_table["Qualify Points"] = race_table["Qualify Points"].fillna(0)
        race_table["Qualify Points"] = race_table["Qualify Points"].astype(int)
    
    return race_table
    
for name, table_df in tables.items():
    
    if name.startswith("Qualify"):
        
        points_column = pd.Series(np.zeros(len(table_df))).add(points_table["qualy_points"].astype(int), fill_value=0)
        table_df["Points"] = points_column.astype(int)
    
    if name.startswith("Race"):
        
        table_df["Time/Retired"] = table_df["Time/Retired"].str.replace('^\s*$', "")
        table_df["Consistency"] = table_df["Consistency"].str.replace('^-$', "")
        
        points_column = pd.Series(np.zeros(len(table_df))).add(points_table["points"].astype(int), fill_value=0)
        table_df["Points"] = points_column.astype(int)
        
        session_num = int(re.findall('(?:Qualify|Race)\s(\d+)\sresult|$', name)[0])
        qualy_table_name = "Qualify result" if session_num==1 and "Qualify result" in tables.keys() else "Qualify {} result".format(session_num)
        previous_race_table_name = "Race {} result".format(session_num - 1)
        if qualy_table_name in tables:
            table_df = merge_qualy_info(table_df, tables[qualy_table_name], add_qualy_points=True)
        elif previous_race_table_name in tables:
            table_df = merge_qualy_info(table_df, tables[previous_race_table_name])
        else:
            table_df["Grid"] = ""    
    
    display(table_df)
    tables[name] = table_df
    

,Pos,Class,Team,Vehicle,Driver,Laps,Best lap,Gap,Points
0,1,,,urd_t5_maures_2018,McLarenTim,6,01:28.597,-,3
1,2,,,urd_t5_maures_2018,PhotonBurst,6,01:28.746,00:00.149,2
2,3,,,urd_t5_aura_2018,gunciz,5,01:29.485,00:00.888,1
3,4,,,urd_t5_aura_2018,MrGrinder,4,01:29.585,00:00.988,0
4,5,,,urd_t5_bayro_2018,Trempale,6,01:29.713,00:01.116,0
5,6,,,urd_t5_bayro_2018,JEMknight657,5,01:30.190,00:01.593,0
6,7,,,urd_t5_aura_2018,Waffles,6,01:30.299,00:01.702,0
7,8,,,urd_t5_bayro_2018,Salzeder,6,01:30.361,00:01.764,0
8,9,,,urd_t5_bayro_2018,NotJames,7,01:30.432,00:01.835,0
9,10,,,urd_t5_aura_2018,xPsychedelix,6,01:30.464,00:01.867,0


,Pos,Class,Team,Vehicle,Driver,Laps,Time/Retired,Best lap,Consistency,Led,Pits,Points,Grid,Qualify Points
0,1,,,urd_t5_maures_2018,McLarenTim,15,22:28.346,01:29.169,99.77%,14,0,40,1,3
1,2,,,urd_t5_maures_2018,PhotonBurst,15,+00:01.347,01:29.111,99.70%,0,0,36,2,2
2,3,,,urd_t5_aura_2018,gunciz,15,+00:18.071,01:30.014,99.60%,0,0,32,3,1
3,4,,,urd_t5_aura_2018,MrGrinder,15,+00:18.866,01:29.700,99.24%,0,0,28,4,0
4,5,,,urd_t5_bayro_2018,JEMknight657,15,+00:19.482,01:29.663,99.35%,0,0,25,6,0
5,6,,,urd_t5_aura_2018,Waffles,15,+00:28.873,01:30.027,98.95%,0,0,22,7,0
6,7,,,urd_t5_aura_2018,gman197002,15,+00:37.582,01:30.864,99.40%,0,0,19,11,0
7,8,,,urd_t5_bayro_2018,Salzeder,15,+00:38.589,01:30.552,99.01%,0,0,16,8,0
8,9,,,urd_t5_bayro_2018,Trempale,15,+00:44.545,01:29.873,97.31%,0,0,13,5,0
9,10,,,urd_t5_aura_2018,xPsychedelix,15,+01:00.145,01:31.127,97.91%,0,0,10,10,0


,Pos,Class,Team,Vehicle,Driver,Laps,Time/Retired,Best lap,Consistency,Led,Pits,Points,Grid
0,1,,,urd_t5_maures_2018,McLarenTim,28,42:14.763,01:29.104,98.64%,23,0,40,1
1,2,,,urd_t5_maures_2018,PhotonBurst,28,+00:28.398,01:29.157,98.72%,4,0,36,2
2,3,,,urd_t5_aura_2018,MrGrinder,28,+00:31.887,01:29.675,98.04%,0,0,32,4
3,4,,,urd_t5_bayro_2018,Trempale,28,+00:41.427,01:30.159,98.46%,0,0,28,9
4,5,,,urd_t5_bayro_2018,JEMknight657,28,+00:41.871,01:29.822,98.20%,0,0,25,5
5,6,,,urd_t5_aura_2018,Waffles,28,+00:50.833,01:30.462,98.27%,0,0,22,6
6,7,,,urd_t5_aura_2018,gman197002,28,+01:09.598,01:30.906,98.19%,0,0,19,7
7,8,,,urd_t5_bayro_2018,Salzeder,28,+01:10.441,01:30.775,97.99%,0,0,16,8
8,9,,,urd_t5_bayro_2018,NotJames,28,+01:11.550,01:30.846,98.36%,0,0,13,11
9,10,,,urd_t5_aura_2018,xPsychedelix,28,+01:13.040,01:30.253,97.97%,0,0,10,10


### Wiki Table Printing

In [602]:
def make_qualy_table(table_name):
    
    table_df = tables[table_name]
    
    lines_buffer = [table_name, qualy_row_0, qualy_row_1]
    
    for i, df_row in table_df.iterrows():
        
        driver = df_row["Driver"]

        driver_info = drivers_table.loc[drivers_table["ign"]==driver]
        assert len(driver_info)==1, "Error looking up {} in drivers_table".format(driver)
        
        position = df_row["Pos"]
        number = driver_info["number"].item()
        flag = driver_info["flag"].item()
        name = driver_info["name"].item()
        team = driver_info["team"].item()
        time = df_row["Best lap"]
        if position==1:
            time = "**{}**".format(time)

        line = qualy_row_2.format(position, number, flag, name, team, time) if i==0 else qualy_row.format(position, number, flag, name, team, time)
        lines_buffer.append(line)
    
    lines_buffer.append(qualy_row_last.format(simresults_url))
    
    print("made {} rows for table: {}".format(len(lines_buffer), table_name))
    return lines_buffer


def make_race_table(table_name, add_qualy_points=False):
    
    table_df = tables[table_name]
    
    lines_buffer = [table_name, race_row_0, race_row_1]
    
    for i, df_row in table_df.iterrows():
        
        driver = df_row["Driver"]

        driver_info = drivers_table.loc[drivers_table["ign"]==driver]
        assert len(driver_info)==1, "Error looking up {} in drivers_table".format(driver)
        
        position = df_row["Pos"]
        number = driver_info["number"].item()
        flag = driver_info["flag"].item()
        name = driver_info["name"].item()
        team = driver_info["team"].item()
        laps = df_row["Laps"]
        points = df_row["Points"]
        timeorretired = df_row["Time/Retired"] if df_row["Time/Retired"] else "DNF"
        grid = df_row["Grid"] if df_row["Grid"]>0 else "DNQ"
        
        if "Qualify Points" in table_df.columns:
            points = str(df_row["Points"] + df_row["Qualify Points"]) + ("^^{}^^".format(grid) if df_row["Qualify Points"]>0 else "")
            
        line = race_row_2.format(position, number, flag, name, team, laps, timeorretired, grid, points) if i==0 else race_row.format(position, number, flag, name, team, laps, timeorretired, grid, points)
        lines_buffer.append(line)
    
    lines_buffer.append(race_row_last.format(simresults_url))
    
    table_df_withtimes = table_df.copy()
    table_df_withtimes["Best lap"] = pd.to_datetime(table_df_withtimes["Best lap"], format="%M:%S.%f", errors='coerce')
    fastest_driver = table_df.loc[table_df_withtimes["Best lap"]==table_df_withtimes["Best lap"].min()].iloc[0]["Driver"]
    fastest_driver_flag = drivers_table.loc[drivers_table["ign"]==fastest_driver].iloc[0]["flag"]
    fastest_time = table_df.loc[table_df["Driver"]==fastest_driver]["Best lap"].item()
    lines_buffer.append(race_row_fastestlap.format(fastest_driver_flag, fastest_driver, fastest_time))
    
    print("made {} rows for table: {}".format(len(lines_buffer), table_name))
    return lines_buffer

In [603]:
tables_lines = {}

for name in table_names:
    if name.startswith("Qualify"):
        tables_lines[name] = make_qualy_table(name)
    elif name.startswith("Race"):
        if "1" in name:
            tables_lines[name] = make_race_table(name, add_qualy_points=True)
        else:
            tables_lines[name] = make_race_table(name)

made 16 rows for table: Qualify result
made 18 rows for table: Race 1 result
made 18 rows for table: Race 2 result


In [604]:
with open(output_file, "w+") as fp:
    for name in table_names:
        if name in tables_lines:
            fp.write("\n".join(tables_lines[name]) + "\n\n")